In [1]:
from fastai.vision.all import *
from fastbook import *

In [2]:
torch.cuda.is_available()

True

## Image loading

In [3]:
path = untar_data(URLs.MNIST)

In [59]:
def loadImages(paths):
    images = [Image.open(p) for p in paths]
    classes = [p.parent.name for p in paths]
    return (images, classes)
    
loadImages([
    path/"training"/"5"/"0.png",
    path/"training"/"9"/"10003.png",
    path/"testing"/"4"/"1059.png"
])

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 ['5', '9', '4'])

In [5]:
import glob

def allImages(path):
    return list(path.glob("**/*.png"))

testImages = allImages(path)
assert len(testImages) > 0
for image in testImages: assert image.name.endswith(".png") 
len(testImages), type(testImages[0])

(70000, pathlib.WindowsPath)

## Model

In [75]:
class Model:
    def __init__(self, imageSize, categoryCount):
        hiddenSize = imageSize[0] * imageSize[1]
        self.w1 = self.__initParams(imageSize[0] * imageSize[1], hiddenSize)
        self.b1 = self.__initParams(hiddenSize)
        
        self.w2 = self.__initParams(hiddenSize, categoryCount)
        self.b2 = self.__initParams(categoryCount)
        
        self.params = [self.w1, self.b1, self.w2, self.b2]
        
    def applyModel(self, batch):
        hiddenLayer1 = F.relu(batch@self.w1 + self.b1)
        return hiddenLayer1@self.w2 + self.b2
    
    def fit(self, lr):
        for p in self.params:
            p.data -= p.grad.data * lr
            p.grad.zero_()
    
    def __initParams(self, *size):
        return (torch.rand(size) * 0.01).requires_grad_()

## Learning preparation

In [7]:
def mse(targetProbabilities, predictedProbabilities):
    return torch.square(targetProbabilities - predictedProbabilities).mean()

testTargets = tensor([1, 0, 1])
test1 = mse(testTargets, tensor([0.9, 0.2, 0.4]))
test2 = mse(testTargets, tensor([0.9, 0.2, 0.5]))
test1, test2

(tensor(0.1367), tensor(0.1000))

In [8]:
def sigmoid(x): return 1/(1+torch.exp(-x))

sigmoid(tensor([-4, -1, 1, 4]))

tensor([0.0180, 0.2689, 0.7311, 0.9820])

In [24]:
def mse_loss(targetProbabilities, predictions):
    return mse(targetProbabilities, predictions.sigmoid())
    
test1 = mse_loss(tensor([1, 0, 1]), tensor([10, -3, 0]))
test2 = mse_loss(tensor([1, 0, 1]), tensor([10, -3, 1]))
test1, test2

(tensor(0.0841), tensor(0.0249))

In [10]:
def calculateAccuracy(targetPredictions, predictions):
    border = 0.5
    return ((predictions > border) == (targetPredictions > border)).float().mean()

calculateAccuracy(tensor([1, 0, 1]), tensor([0.2, 0, 0.61]))

tensor(0.6667)

In [11]:
class BatchLoader:
    def __init__(self, images, batchSize):
        self.images = images.copy()
        random.shuffle(self.images)
        self.batchSize = batchSize
        self.nextBatch = 0 
    def nextBatch(self):
        batchStartIndex = min(self.batchSize * self.nextBatch, len(self.images))
        batchEndIndex = min(startBatchFrom + self.batchSize, len(self.images))
        if batchStartIndex == batchEndIndex:
            return None
        else:
            return self.images[batchStartIndex:batchEndIndex]

In [12]:
class BatchLoader:
    def __init__(self, items, batchSize):
        self.items = items.copy()
        random.shuffle(self.items)
        self.batchSize = batchSize
        self.nextBatch = 0 
    def getNextBatch(self):
        batchStartIndex = min(self.batchSize * self.nextBatch, len(self.items))
        batchEndIndex = min(batchStartIndex + self.batchSize, len(self.items))
        if batchStartIndex == batchEndIndex:
            return None
        else:
            self.nextBatch += 1
            return self.items[batchStartIndex:batchEndIndex]
       
    
testItems = [1, 2, 3, 4, 5, 6, 7]
testLoader = BatchLoader(testItems, 4)
receivedItems = testLoader.getNextBatch() + testLoader.getNextBatch()
receivedItems.sort()
assert testItems == receivedItems, "received aren't the same as tests " + receivedItems
assert testLoader.getNextBatch() == None

In [47]:
def loadBatch(paths):
    images = [tensor(Image.open(p)).view(-1) for p in paths]
    classes = [p.parent.name for p in paths]
    return (torch.stack(images).float()/255, classes)
    
testBatchLoader = BatchLoader(allImages(path/"training"), 100)
trainingImages, trainingClasses = loadBatch(testBatchLoader.getNextBatch())
assert len(trainingImages) == len(trainingClasses)

In [14]:
def numberToPrediction(number):
    return [1 if i == number else 0 for i in range(10)]

def targetPredictions(classes):
    return tensor([numberToPrediction(int(c)) for c in classes])

targetPredictions(["0", "5", "9"])

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

### The learning loop

In [62]:
def validateOneBatch(model, batch):
    images, classes = loadBatch(batch)
    predictions = model.applyModel(images).sigmoid()
    target = targetPredictions(classes)
    return calculateAccuracy(target, predictions)

validateOneBatch(Model((28,28), 10), allImages(path/"testing/4")[:10])

tensor(0.1000)

In [63]:
def validateEpoach(model, validationSet, batchSize):
    batchLoader = BatchLoader(validationSet, batchSize)
    batch = batchLoader.getNextBatch()
    accuracies = []
    while batch != None:
        accuracies.append(validateOneBatch(model, batch))
        batch = batchLoader.getNextBatch()
    return round(torch.stack(accuracies).mean().item(), 4)

validateEpoach(Model((28,28), 10), allImages(path/"testing"), 10000)

0.1

In [44]:
def fitOneBatch(model, batch, lr):
    traingingImages, trainingClasses = loadBatch(batch)
    predictions = model.applyModel(traingingImages).sigmoid()
    #print("predictions " + str(predictions))
    target = targetPredictions(trainingClasses)
    #print("target " + str(target))
    loss = mse(target, predictions)
    loss.backward()
    #print("Grad w1 " + str(model.w1.grad.mean()) + ", b1 " + str(model.b1.grad.mean()))
    model.fit(lr)
    print("Loss: " + str(loss.item()))

fitOneBatch(Model((28, 28), 10), allImages(path/"training/4")[:10], 1.)

Loss: 0.34308311343193054


In [68]:
def trainEpoach(model, trainingSet, validationSet,  batchSize, lr):
    batchLoader = BatchLoader(trainingSet, batchSize)
    batch = batchLoader.getNextBatch()
    while batch != None:
        fitOneBatch(model, batch, lr)
        batch = batchLoader.getNextBatch()
    accuracy = validateEpoach(model, validationSet, batchSize)
    print("Accuracy " + str(accuracy))
    
model = Model((28, 28), 10)    
trainEpoach(model, allImages(path/"training"), allImages(path/"testing"), 10000, 1.)

Loss: 0.3680417537689209
Loss: 0.18425633013248444
Loss: 0.12630914151668549
Loss: 0.10910668224096298
Loss: 0.101543128490448
Loss: 0.09696771204471588
Accuracy 0.9


In [ ]:
model = Model((28, 28), 10)
for e in range(1, 6):
    trainEpoach(model, allImages(path/"training"), allImages(path/"testing"), 10000, 1. / e)

Loss: 0.6824244260787964
Loss: 0.12574312090873718
Loss: 0.0948319360613823
Loss: 0.09444957971572876
Loss: 0.09415306895971298
Loss: 0.09382064640522003
Accuracy 0.9
Loss: 0.0937221497297287
Loss: 0.09368693083524704
Loss: 0.09361375868320465
Loss: 0.0935286357998848
Loss: 0.09349168837070465
Loss: 0.0934002697467804
Accuracy 0.9
Loss: 0.09346888959407806
Loss: 0.09342165291309357
Loss: 0.0934908390045166
Loss: 0.09345271438360214
Loss: 0.09324082732200623
Loss: 0.09333856403827667
Accuracy 0.9
